In [1]:
import pandas as pd
import numpy as np
import spacy
from spacy.lang.en import English
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as smote_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn import impute

import datetime as dt

import joblib
from sklearn.model_selection import GridSearchCV, cross_val_score
import xgboost as xgb

from sklearn import set_config
set_config(display='diagram')

/Users/josh-mantovani/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/josh-mantovani/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/josh-mantovani/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/josh-mantovani/anaconda3/lib/python3.7/site-pack

In [3]:
df = pd.read_json('./../data/data.json')
df

,acct_type,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,...,ticket_types,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state
0,fraudster_event,1266062400,3852,5,US,USD,0.0,"<p><a href=""http://s432.photobucket.com/albums...",gmail.com,1262739706,...,"[{'event_id': 527017, 'cost': 25.0, 'availabil...",36,1259613950,1,717 Washington Avenue,US,25.777471,-80.133433,INK Nightclub - South Beach,FL
1,premium,1296720000,3499,0,US,USD,1.0,"<p>Join us for a quick, one-night, community-b...",ruf.org,1293832670,...,"[{'event_id': 786878, 'cost': 35.0, 'availabil...",149,1280942776,3,,US,32.776566,-79.930922,"The Charleston, SC area",SC
2,premium,1296172800,2601,8,US,USD,1.0,"<h3><span class=""subcategory""><strong>Teacher ...",pvsd.k12.ca.us,1291090956,...,"[{'event_id': 787337, 'cost': 93.51, 'availabi...",214,1272559388,3,10100 Pioneer Blvd Suite 100,US,33.944201,-118.080419,Los Angeles County Office of Education,CA
3,premium,1388966400,12347,6,IE,EUR,1.0,"<p style=""margin-bottom: 1.3em; padding-bottom...",irishtabletennis.com,1360681570,...,"[{'event_id': 885645, 'cost': 25.0, 'availabil...",889,1283870102,3,,None,NaN,NaN,None,None
4,premium,1297900800,2417,11,US,USD,0.0,<p>Writers and filmmakers need to understand t...,artsandbusinesscouncil.org,1291994666,...,"[{'event_id': 1114349, 'cost': 150.0, 'availab...",35,1288984065,3,One Marina Park Drive,US,42.353848,-71.044276,Fish & Richardson,MA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14332,fraudster_event,1361689200,667,0,US,USD,0.0,<p>Celebrates this wonderful holiday with the ...,yahoo.com,1360297993,...,"[{'event_id': 5456442, 'cost': 45.0, 'availabi...",0,1360297993,1,6100 North Charles Street,US,39.373780,-76.629921,Elkridge Club,MD
14333,premium,1365566400,5812,5,US,USD,1.0,"<table style=""width: 600px;"" border=""0"" cellsp...",me.com,1360367042,...,"[{'event_id': 5465112, 'cost': 90.0, 'availabi...",1374,1241625346,4,1775 E Mission Bay Dr,US,32.778906,-117.209791,Hilton San Diego Resort and Spa (Hilton Missio...,CA
14334,premium,1368781200,557,13,,USD,0.0,<p>What a way to start off your Mother's Day l...,yahoo.com,1360600330,...,"[{'event_id': 5465732, 'cost': 20.0, 'availabi...",84,1353386971,4,5991 Bullard Road,US,30.041819,-89.957130,Fleur De Lis Ballroom NUMBER 2,LA
14335,tos_warn,1361361600,10873,8,US,USD,0.0,"<p style=""font-family: 'Helvetica Neue', Helve...",velvetlist.com,1360376285,...,"[{'event_id': 5465810, 'cost': 79.33, 'availab...",812,1290204991,3,2297 Cedar Ave,US,40.862283,-73.911363,Salsa con fuego,NY


In [10]:
y = df.pop('fraud')
X = df.copy()

In [11]:
X.iloc[:, 1]

0        <p><a href="http://s432.photobucket.com/albums...
1        <p>Join us for a quick, one-night, community-b...
2        <h3><span class="subcategory"><strong>Teacher ...
3        <p style="margin-bottom: 1.3em; padding-bottom...
4        <p>Writers and filmmakers need to understand t...
                               ...                        
14332    <p>Celebrates this wonderful holiday with the ...
14333    <table style="width: 600px;" border="0" cellsp...
14334    <p>What a way to start off your Mother's Day l...
14335    <p style="font-family: 'Helvetica Neue', Helve...
14336              thaimassageali osman192 Pembroke Street
Name: description, Length: 14337, dtype: object

In [4]:
my_stops = ['0pt', '1pt', '2pt', '4pt', '10pt', '12pt', '14pt', '15pt',
            '0px', '1px', '2px', '4px', '10px', '12px', '14px', '15px',
            'rgb', '255', '0', 'li', 'div', 'u', 'b', '0001pt', '39', '51'
            'meta', 'font', 'size', 'arial', 'nbsp', 'align', 'justify',
            'href', 'style', 'quot', 'msonormal', 'serif', 'text', 'ldquo',
            'rdquo', 'height', 'text', 'mso', 'san', 'margin', 'class', 'tab',
            'roman', 'times', 'http', 'www', 'html', 'background', 'pad',
            'bidi', 'color', 'bidi', 'san', 'rsquo', 'br', 'spin', 'letter',
            'spacing', 'space', 'hyphenate', 'place', 'line', 'placename',
            'placetype', 'border', 'box', 'normal', 'com', 'url', 'link',
            'publish', 'lsdexception', '00', '000', '000000', 'river',
            'family', 'water', 'boat', 'stay', 'helvetica', 'st', 'inherit',
            'width', 'false', 'face', 'non', '51', 'say', 'raft', 'rapid',
            'year', '1', '2', '3', 'rescue', 'true', 'paddle', 'w',
            'lock', 'priority', 'accent', 'semihidden', 'unhidewhenused',
            'table', 'list', 'lock', 'semihidden', 'amp', 'bt', 'grid',
            'layout', 'mode', 'narrative', 'initial', 'variant', 'weight',
            'outline', 'baseline', 'datum', 'vertical', 'leave', 'image',
            'max', 'position', 'display', '68', 'https', 'right', 'ligature',
            'stockticker', '08', '11', '06', '12', 'pa', 'source', '11pt',
            'large', 'march', 'tira', 'niyhwk', 'tcenter', 'posr', 'jim',
            'georgia', 'lucas', 'posr', 'mark', 'get', 'rock', 'be', 'kayker',
            'time', 'ndn', 'thumbtitle', 'thumbnail', 'sliderthumbnailoverlay',
            'neacato', '07', 'witness', 'stockticker', '4', '5', '6', '7',
            'jpg', '300w', 'neue', 'lucida', 'header', 'segoe', 'byline',
            'at4', '75em', '400', '1rem', 'and', 'let', 'near', 'new', '\xa0 ',
            'h1', 'eacute', 'center', '13px', 'rd', 'de', 'center' '800000',
            '\x1f', 'ltr', 'verdana', '-PRON-', '17', '4c4c4c', '17 40625px',
            '11px 17', 'lt', '11px', 'p', 'span', ' ']

seperators = ['.', ';', ':', '/', '&', '=', '(', ')', '-', ',', '>', '<', '_',
              '{', '}']

htmls = ['\\', '\r', '\n', '\t']

spaces = [' '*i for i in range(1, 6)]

In [33]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.20)
X_train

,body_length,description,fb_published,has_analytics,has_logo,listed,name_length,num_payouts,org_facebook,org_twitter,...,payout_type__,payout_type__ACH,payout_type__CHECK,pop_country,country_is_null,venue_country_is_null,country_is_venue,avg_ticket_price,tot_ticket_available,avg_cost_per_ticket
4558,755,"<p> </p>\r\n<p><font size=""3"" color=""#000000"">...",0,0,1,0,45,0,0.0,0.0,...,0,1,0,0,0,0,1,5.00,100,0.050000
11543,95,<p>Testing</p>\r\n<p> </p>\r\n<p>TESTING</p>\r...,0,0,1,0,13,1,12.0,0.0,...,0,1,0,1,0,0,1,15.20,3,5.066667
9296,1607,"<p><span style=""color: #333333; font-family: '...",0,0,1,1,40,4,0.0,0.0,...,0,1,0,1,0,0,1,13.18,100,0.131800
12022,3084,"<p><font face=""verdana, geneva"" size=""3"" color...",1,0,1,1,137,0,0.0,0.0,...,0,1,0,1,0,0,1,21.91,35,0.626000
3397,6654,"<p style=""text-align: center; line-height: nor...",1,0,1,1,28,0,28.0,0.0,...,0,1,0,1,0,0,1,6.63,500,0.013260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9904,11253,"<p><strong>Building a Smart Future</strong>, t...",0,0,1,1,23,16,12.0,12.0,...,0,1,0,1,0,0,1,27.55,103,0.267476
11272,1511,<p>Join Bill Prim for his Winter Fundraiser ho...,0,0,0,1,46,0,14.0,14.0,...,0,0,1,1,0,0,1,24.12,3000,0.008040
6770,28,<p>Vendors Spaces 2 left</p>Vendors Insurance<...,0,0,1,1,17,0,0.0,0.0,...,0,1,0,0,0,1,0,1250.00,2,625.000000
1126,4877,"<p><span style=""color: #333333; line-height: 1...",0,0,1,1,63,0,0.0,0.0,...,0,1,0,1,0,0,1,25.00,150,0.166667


In [30]:
nlp = English()
nlp.Defaults.stop_words |= set(my_stops)

In [35]:
from spacy.lemmatizer import Lemmatizer
from spacy.lookups import Lookups
import re

lemma = Lemmatizer(Lookups())

In [36]:
# lemma.lookup('Test sentences, Im running')

'Test sentences, Im running'

In [37]:
# tokenizer = nlp.Defaults.create_tokenizer()
# tokens = tokenizer(X_train.iloc[0, 1])
# for i in tokens:
#     print(i)

<
p
>
 
<
/p
>


<
p><font
size="3
"
color="#000000">Nicole
Bridger
Designs
is
 
hosting
the
next
SVI
Hollyhock
gathering
at
our
new
green
boutique
in
Kitsilano
.
We
have
great
food
and
drink
sponsors
,
so
all
you
need
to
bring
is
you!</font></p
>


<
p><font
size="3
"
color="#000000
"
>
 
<
/font></p
>


<
p><font
size="3
"
color="#000000">When
:
June
29th
,
6:30pm
<
br
/>Where
:
2151
W4th
Ave</font></p
>


<
p><br
/><br
/><font
size="3
"
color="#000000">We
will
be
having
a
Marketplace
of
Ideas
at
7:30pm
,
if
you
want
to
be
a
part
of
it
please
rsvp
to
<
/font><a
href="mailto
:
kimburden@nicolebridger.com"><span
style="text
-
decoration
:
underline;"><font
size="3
"
color="#000000">kimburden@nicolebridger.com</font></span></a></p
>


<
p><font
size="3
"
color="#000000
"
>
 
<
/font></p
>


<
p
>
 
<
/p
>
Nicole
Bridger
hosts
SVI
Hollyhock
Gathering
Nicole
Bridger


In [40]:
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    text = re.sub('[\W]+', '', text.lower())
    return text

def tokenize_and_lemmatize2(text):
    nlp = English()
    nlp.Defaults.stop_words |= set(my_stops)
    tokenizer = nlp.Defaults.create_tokenizer()
    tokens = tokenizer(text)
    lemmatized_tokens = []
    for token in tokens:
        if not token.is_stop:
            processed_token = preprocessor(token.lemma_)
            if processed_token != '':
                lemmatized_tokens.append(processed_token)
    return lemmatized_tokens

tokenize_and_lemmatize2('test sentences im running run   <p><span style="font">')

['test', 'sentences', 'be', 'running', 'run', 'pspan', 'stylefont']

In [39]:
def tokenize_and_lemmatize(text):
    text = str(text).lower()
    for sep in seperators:
        text = text.replace(sep, ' ')
    for html in htmls:
        text = text.replace(html, '')
    for space in spaces:
        text = text.replace(space, ' ')
    doc = nlp(text)
    words = []
    for token in doc:
        lemma = token.lemma_
        if not token.is_stop and not token.is_punct and lemma not in my_stops:
            words.append(lemma)
    return words

In [49]:
vectorizer = TfidfVectorizer(ngram_range=(1,2),
                            max_df=0.9,
                            max_features=10000,
                            token_pattern=None,
                            tokenizer=tokenize_and_lemmatize2)
vectorizer.fit(X_train.iloc[:, 1])


KeyboardInterrupt: 

In [152]:
vectorizer = CountVectorizer(ngram_range=(1, 3),
                            max_df=.80,
                            max_features=10000,
                            token_pattern=None,
                            tokenizer=tokenize_and_lemmatize2)
vectorizer.fit_transform(X_train.iloc[:, 1])

KeyboardInterrupt: 

In [ ]:
vectorized_train_descriptions = vectorizer.transform(X_train.iloc[:, 1])
vectorized_test_descriptions = vectorizer.transform(X_test.iloc[:, 1])

In [ ]:
bayes = MultinomialNB()
bayes.fit(vectorized_train_descriptions, y_train)

In [ ]:
train_predictions = bayes.predict(vectorized_train_descriptions)
test_predictions = bayes.predict(vectorized_test_descriptions)


In [ ]:
f1_score(y_train, train_predictions)


In [ ]:
f1_score(y_test, test_predictions)

In [132]:
def nlp_transformer(X, y=None):
    predictions = bayes.predict_proba(X)
    return predictions[:, 1]

bayes_transformer = preprocessing.FunctionTransformer(nlp_transformer)

In [133]:
nlp_transformer = make_pipeline(
    vectorizer, bayes_transformer
)
nlp_transformer

Pipeline(steps=[('countvectorizer',
                 CountVectorizer(max_df=0.8, max_features=10000,
                                 ngram_range=(1, 3), token_pattern=None,
                                 tokenizer=<function tokenize_and_lemmatize at 0x7fd8e86ced40>)),
                ('functiontransformer',
                 FunctionTransformer(func=<function nlp_transformer at 0x7fd8ba881ef0>))])

In [142]:
num_pipe = make_pipeline(
    std
)
num_pipe

Pipeline(steps=[('standardscaler', StandardScaler())])

In [139]:
std = preprocessing.StandardScaler().fit(X_train.drop('description', axis=1))
std

StandardScaler()

In [150]:
transformer = make_column_transformer(
    (nlp_transformer, ['description']),
    (num_pipe, [col for col in X_train.columns if col != 'description'])
)

transformer
# pipe = Pipeline(steps=[
#     ('transformer', transformer),
#     ('cls', RandomForestClassifier(n_jobs=-1))
# ])
# pipe

ColumnTransformer(transformers=[('pipeline-1',
                                 Pipeline(steps=[('countvectorizer',
                                                  CountVectorizer(max_df=0.8,
                                                                  max_features=10000,
                                                                  ngram_range=(1,
                                                                               3),
                                                                  token_pattern=None,
                                                                  tokenizer=<function tokenize_and_lemmatize at 0x7fd8e86ced40>)),
                                                 ('functiontransformer',
                                                  FunctionTransformer(func=<function nlp_transformer at 0x7fd8ba881ef0>))]),
                                 ['description']),
                                ('pipeline-2',...
                                  'name_length', 'num_payouts', 'org_facebook',
                                  'org_twitter', 'show_map', 'user_age',
                                  'time_to_event', 'event_length',
                                  'channels__0', 'channels__4', 'channels__5',
                                  'channels__6', 'channels__7', 'channels__8',
                                  'channels__9', 'channels__10', 'channels__11',
                                  'channels__12', 'channels__13',
                                  'delivery_method__0.0',
                                  'delivery_method__1.0',
                                  'delivery_method__3.0', 'user_type__1',
                                  'user_type__2', 'user_type__3', ...])])

In [151]:
transformer.transform(X_train)

NotFittedError: This ColumnTransformer instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [126]:
train_predictions_proba = bayes.predict_proba(vectorized_train_descriptions)
test_predictions_proba = bayes.predict_proba(vectorized_test_descriptions)


In [130]:
train_predictions_proba

array([[1.14801668e-005, 9.99988520e-001],
       [1.00000000e+000, 2.87857265e-041],
       [1.00000000e+000, 1.77150599e-022],
       ...,
       [1.00000000e+000, 4.10404629e-135],
       [1.00000000e+000, 1.46191327e-057],
       [7.98131650e-005, 9.99920187e-001]])

In [131]:
def remove_description_and_add_proba(data, probas):
    d = data.copy()
    d.drop('description', axis=1, inplace=True)
    d['description_proba'] = probas
    return d

X_train_processed = remove_description_and_add_proba(X_train, train_predictions_proba[:, 1])
X_test_processed = remove_description_and_add_proba(X_test, test_predictions_proba[:, 1])
X_train_processed

,body_length,fb_published,has_analytics,has_logo,listed,name_length,num_payouts,org_facebook,org_twitter,show_map,...,payout_type__ACH,payout_type__CHECK,pop_country,country_is_null,venue_country_is_null,country_is_venue,avg_ticket_price,tot_ticket_available,avg_cost_per_ticket,description_proba
2412,1207,1,0,1,1,20,0,18.0,0.0,1,...,1,0,1,0,0,1,0.000000,1,0.000000,9.999885e-01
12359,4965,0,0,1,1,34,62,0.0,0.0,1,...,1,0,1,0,0,1,141.510000,50,2.830200,2.878573e-41
10270,967,0,0,0,1,46,43,0.0,0.0,0,...,0,1,1,0,1,0,13.260000,2950,0.004495,1.771506e-22
9131,1735,0,0,0,0,51,8,28.0,6.0,0,...,1,0,0,0,1,0,38.000000,800,0.047500,1.136375e-21
6573,4242,0,0,1,1,73,15,6.0,0.0,0,...,0,1,0,1,1,0,22.630000,500,0.045260,1.135094e-39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9899,2759,0,0,1,1,57,32,12.0,14.0,1,...,1,0,1,0,0,1,99.225000,24,4.134375,7.610799e-17
13146,3896,0,0,1,1,39,0,0.0,0.0,1,...,1,0,1,0,0,1,273.333333,425,0.643137,6.928159e-18
1976,900,0,0,1,1,24,143,13.0,14.0,1,...,0,1,1,0,0,1,60.430000,9,6.714444,4.104046e-135
1526,340,1,0,0,1,90,696,0.0,0.0,1,...,1,0,1,0,0,1,86.885000,2000,0.043443,1.461913e-57


In [135]:
X_train_processed[X_train_processed['description_proba']>= .70]

,body_length,fb_published,has_analytics,has_logo,listed,name_length,num_payouts,org_facebook,org_twitter,show_map,...,payout_type__ACH,payout_type__CHECK,pop_country,country_is_null,venue_country_is_null,country_is_venue,avg_ticket_price,tot_ticket_available,avg_cost_per_ticket,description_proba
2412,1207,1,0,1,1,20,0,18.0,0.0,1,...,1,0,1,0,0,1,0.00,1,0.000000,0.999989
3501,1102,0,0,1,1,55,0,0.0,0.0,1,...,1,0,1,0,0,1,5.00,145,0.034483,0.978436
8782,907,1,0,1,0,29,0,22.0,10.0,1,...,1,0,1,0,0,1,8.75,15,0.583333,0.998314
3528,918,0,0,1,1,22,0,0.0,0.0,0,...,1,0,1,0,0,0,10.00,400,0.025000,0.993436
721,2053,0,0,1,1,25,0,0.0,0.0,1,...,1,0,1,0,0,1,67.63,450,0.150289,0.999952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3952,1481,0,0,0,1,30,0,0.0,0.0,1,...,1,0,0,0,0,1,54.00,130,0.415385,1.000000
2334,488,1,0,1,1,11,0,13.0,14.0,1,...,1,0,1,0,0,1,10.00,20,0.500000,0.732793
3991,140,0,0,0,1,8,0,0.0,0.0,1,...,1,0,1,0,0,1,50.00,30,1.666667,0.998225
9216,511,0,0,0,0,29,0,0.0,0.0,1,...,1,0,1,0,0,1,25.00,200,0.125000,1.000000


In [141]:
sm = SMOTE(n_jobs=-1)
X_train_sm, y_train_sm = sm.fit_resample(X_train_processed, y_train)


In [159]:
unique_values, counts = np.unique(y_train_sm, return_counts=True)
for u, c in zip(unique_values, counts):
    print(u, c)

0 10430
1 10430


In [183]:
pipe = Pipeline(steps=[
    ('z_scorer', preprocessing.StandardScaler()),
    ('poly', preprocessing.PolynomialFeatures(degree=5)),
    ('cls', LogisticRegressionCV(n_jobs=-1))
])
pipe

Pipeline(steps=[('z_scorer', StandardScaler()),
                ('poly', PolynomialFeatures(degree=5)),
                ('cls', LogisticRegressionCV(n_jobs=-1))])

In [ ]:
pipe.fit(X_train_processed, y_train)

In [ ]:
train_predictons = pipe.predict(X_train_processed)
test_predictions = pipe.predict(X_test_processed)

f1_score(y_train, train_predictions)

In [174]:
np.unique(train_predictions)

array([0])

In [160]:
lr = LogisticRegression(n_jobs=-1)
lr.fit(X_train_processed, y_train)

LogisticRegression(n_jobs=-1)

In [161]:
train_predictions = lr.predict(X_train_processed)
test_predictions = lr.predict(X_test_processed)

In [162]:
f1_score(y_train, train_predictions)

0.0

In [163]:
f1_score(y_test, test_predictions)

0.0

In [164]:
lr.coef_

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0.]])

In [149]:
rf = RandomForestClassifier()
rf.fit(X_train_sm, y_train_sm)

RandomForestClassifier()

In [150]:
f1_score(rf.predict(X_train_processed), y_train)

1.0

In [151]:
f1_score(rf.predict(X_test_processed), y_test)

0.8447204968944099

In [180]:
importances = rf.feature_importances_
features = X_train_processed.columns

order = np.argsort(importances)[::-1]

for f, i in zip(features[order], importances[order]):
    print(f, i)

description_proba 0.23920716782928314
time_to_event 0.11281132179346955
num_payouts 0.09504420385477888
user_age 0.07299577341548477
delivery_method__1.0 0.045620705688207794
org_twitter 0.03850225150282613
org_facebook 0.038394744483960894
country_is_venue 0.03259004684854745
user_type__3 0.0321790738722299
avg_ticket_price 0.031527492580378055
payout_type__CHECK 0.031058920729150747
has_logo 0.02563549972387795
payout_type__ACH 0.02375911295154319
user_type__4 0.019960702899812402
body_length 0.016850285171078054
avg_cost_per_ticket 0.016287039610356893
event_length 0.015288395850959003
payout_type__ 0.015103137897068374
tot_ticket_available 0.012536156451135004
name_length 0.01131571657665095
delivery_method__0.0 0.009606621759209562
pop_country 0.00651836916565128
channels__8 0.006133298507958903
fb_published 0.006065343807240015
currency__USD 0.0053479506586213265
user_type__1 0.00506975481226163
show_map 0.004884062092363113
channels__11 0.004190497740608432
currency__CAD 0.00377

In [160]:
pd.options.display.max_columns = None
pd.read_json('http://galvanize-case-study-on-fraud.herokuapp.com/data_point', orient='index').transpose()

,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,event_end,event_published,event_start,fb_published,gts,has_analytics,has_header,has_logo,listed,name,name_length,num_order,num_payouts,object_id,org_desc,org_facebook,org_name,org_twitter,payee_name,payout_type,previous_payouts,sale_duration,sale_duration2,show_map,ticket_types,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state
0,1357207200,2745,5,US,USD,0,"<p><span style=""color: #333333; font-family: '...",yahoo.com,1355248406,1356775200,1.35525e+09,1356753600,0,300.04,0,0,1,y,"PRE-NEW YEARS REVOLUTION, THE END PARTY",39,16,2,5008750,,0,JDM MEDIA GROUP,0,Justin Bing / JDM Media Group LLC.,CHECK,"[{'address': '5633 E 14th St', 'amount': 20.0,...",17,17,1,"[{'availability': 1, 'cost': 10.0, 'event_id':...",315,1328078383,1,DV8 NIGHTCLUB,US,32.2371,-110.867,,AZ


In [165]:
import time
df = pd.read_json('http://galvanize-case-study-on-fraud.herokuapp.com/data_point', orient='index').transpose()
for _ in range(20):
    display(df)
    time.sleep(45)
    new_datapoint = pd.read_json('http://galvanize-case-study-on-fraud.herokuapp.com/data_point', orient='index').transpose()
    df = df.append(new_datapoint, ignore_index=True)
    

,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,event_end,event_published,event_start,fb_published,gts,has_analytics,has_header,has_logo,listed,name,name_length,num_order,num_payouts,object_id,org_desc,org_facebook,org_name,org_twitter,payee_name,payout_type,previous_payouts,sale_duration,sale_duration2,show_map,ticket_types,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state
0,1346626800,82,0,US,USD,0,<p>KW Luxury Homes Summit - Lunch and Cruise o...,kwspokane.com,1342719392,1346194800,1.34272e+09,1343413800,0,602.14,0,0,1,n,KW Luxury Homes - lunch and cruise registration,47,8,3,3829206,,11,Keller Williams Northwest Region,7,Keller Williams Realty - c/o Mike Pfau,CHECK,"[{'address': '3840 East Ray Road', 'amount': 3...",8,8,1,"[{'availability': 1, 'cost': 25.0, 'event_id':...",932,1262197475,3,(Located on the north end of Lake Union),US,47.6511,-122.33,Waterways Cruises HomePort,WA


,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,event_end,event_published,event_start,fb_published,gts,has_analytics,has_header,has_logo,listed,name,name_length,num_order,num_payouts,object_id,org_desc,org_facebook,org_name,org_twitter,payee_name,payout_type,previous_payouts,sale_duration,sale_duration2,show_map,ticket_types,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state
0,1346626800,82,0,US,USD,0,<p>KW Luxury Homes Summit - Lunch and Cruise o...,kwspokane.com,1342719392,1346194800,1.34272e+09,1343413800,0,602.14,0,0,1,n,KW Luxury Homes - lunch and cruise registration,47,8,3,3829206,,11,Keller Williams Northwest Region,7,Keller Williams Realty - c/o Mike Pfau,CHECK,"[{'address': '3840 East Ray Road', 'amount': 3...",8,8,1,"[{'availability': 1, 'cost': 25.0, 'event_id':...",932,1262197475,3,(Located on the north end of Lake Union),US,47.6511,-122.33,Waterways Cruises HomePort,WA
1,1366936200,2090,0,US,USD,1,"<p style=""font-family: 'Helvetica Neue', Helve...",rebuildingexchange.org,1362440224,1366504200,1.36244e+09,1366493400,0,765,0,0,1,y,WOOD BENCH - MAKE IT/ TAKE IT ONE DAY CLASS,43,9,59,5695572,<p>Our mission is to create a market for recla...,18,Rebuilding Exchange,10,,ACH,"[{'address': '', 'amount': 130.0, 'country': '...",47,47,1,"[{'availability': 1, 'cost': 79.76, 'event_id'...",507,1318631490,3,1740 W Webster Ave,US,41.9215,-87.6713,Rebuilding Exchange,IL


,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,event_end,event_published,event_start,fb_published,gts,has_analytics,has_header,has_logo,listed,name,name_length,num_order,num_payouts,object_id,org_desc,org_facebook,org_name,org_twitter,payee_name,payout_type,previous_payouts,sale_duration,sale_duration2,show_map,ticket_types,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state
0,1346626800,82,0,US,USD,0,<p>KW Luxury Homes Summit - Lunch and Cruise o...,kwspokane.com,1342719392,1346194800,1.34272e+09,1343413800,0,602.14,0,0,1,n,KW Luxury Homes - lunch and cruise registration,47,8,3,3829206,,11,Keller Williams Northwest Region,7,Keller Williams Realty - c/o Mike Pfau,CHECK,"[{'address': '3840 East Ray Road', 'amount': 3...",8,8,1,"[{'availability': 1, 'cost': 25.0, 'event_id':...",932,1262197475,3,(Located on the north end of Lake Union),US,47.6511,-122.33,Waterways Cruises HomePort,WA
1,1366936200,2090,0,US,USD,1,"<p style=""font-family: 'Helvetica Neue', Helve...",rebuildingexchange.org,1362440224,1366504200,1.36244e+09,1366493400,0,765,0,0,1,y,WOOD BENCH - MAKE IT/ TAKE IT ONE DAY CLASS,43,9,59,5695572,<p>Our mission is to create a market for recla...,18,Rebuilding Exchange,10,,ACH,"[{'address': '', 'amount': 130.0, 'country': '...",47,47,1,"[{'availability': 1, 'cost': 79.76, 'event_id'...",507,1318631490,3,1740 W Webster Ave,US,41.9215,-87.6713,Rebuilding Exchange,IL
2,1356037200,4635,8,US,USD,1,"<span class=""vevent""><span class=""description""...",savings.com,1352056863,1355605200,0,1355594400,0,160,0,0,0,y,Colorado Grocery Savings and Couponing Workshop,47,23,53,4749437,,7,Savings Nation,8,,ACH,"[{'address': '', 'amount': 178.39, 'country': ...",None,41,1,"[{'availability': 1, 'cost': 13.92, 'event_id'...",250,1330452900,3,5955 South Holly Street,US,39.6089,-104.924,Koelbel Public Library,CO


,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,event_end,event_published,event_start,fb_published,gts,has_analytics,has_header,has_logo,listed,name,name_length,num_order,num_payouts,object_id,org_desc,org_facebook,org_name,org_twitter,payee_name,payout_type,previous_payouts,sale_duration,sale_duration2,show_map,ticket_types,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state
0,1346626800,82,0,US,USD,0,<p>KW Luxury Homes Summit - Lunch and Cruise o...,kwspokane.com,1342719392,1346194800,1.34272e+09,1343413800,0,602.14,0,0,1,n,KW Luxury Homes - lunch and cruise registration,47,8,3,3829206,,11,Keller Williams Northwest Region,7,Keller Williams Realty - c/o Mike Pfau,CHECK,"[{'address': '3840 East Ray Road', 'amount': 3...",8,8,1,"[{'availability': 1, 'cost': 25.0, 'event_id':...",932,1262197475,3,(Located on the north end of Lake Union),US,47.6511,-122.33,Waterways Cruises HomePort,WA
1,1366936200,2090,0,US,USD,1,"<p style=""font-family: 'Helvetica Neue', Helve...",rebuildingexchange.org,1362440224,1366504200,1.36244e+09,1366493400,0,765,0,0,1,y,WOOD BENCH - MAKE IT/ TAKE IT ONE DAY CLASS,43,9,59,5695572,<p>Our mission is to create a market for recla...,18,Rebuilding Exchange,10,,ACH,"[{'address': '', 'amount': 130.0, 'country': '...",47,47,1,"[{'availability': 1, 'cost': 79.76, 'event_id'...",507,1318631490,3,1740 W Webster Ave,US,41.9215,-87.6713,Rebuilding Exchange,IL
2,1356037200,4635,8,US,USD,1,"<span class=""vevent""><span class=""description""...",savings.com,1352056863,1355605200,0,1355594400,0,160,0,0,0,y,Colorado Grocery Savings and Couponing Workshop,47,23,53,4749437,,7,Savings Nation,8,,ACH,"[{'address': '', 'amount': 178.39, 'country': ...",None,41,1,"[{'availability': 1, 'cost': 13.92, 'event_id'...",250,1330452900,3,5955 South Holly Street,US,39.6089,-104.924,Koelbel Public Library,CO
3,1353801600,291,8,AU,AUD,1,<p>This workshop explores working with young p...,wodongatafe.edu.au,1351212497,1353369600,1.35121e+09,1353344400,0,366.57,0,None,1,y,Working with Young Parents and their Children ...,55,6,19,4658497,,0,Wodonga Institute of TAFE,0,,ACH,"[{'address': '', 'amount': 261.69, 'country': ...",25,25,1,"[{'availability': 1, 'cost': 40.37, 'event_id'...",158,1337565408,3,286 Napier St,AU,-36.7487,144.292,Lakeview Resort,VIC


,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,event_end,event_published,event_start,fb_published,gts,has_analytics,has_header,has_logo,listed,name,name_length,num_order,num_payouts,object_id,org_desc,org_facebook,org_name,org_twitter,payee_name,payout_type,previous_payouts,sale_duration,sale_duration2,show_map,ticket_types,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state
0,1346626800,82,0,US,USD,0,<p>KW Luxury Homes Summit - Lunch and Cruise o...,kwspokane.com,1342719392,1346194800,1.34272e+09,1343413800,0,602.14,0,0,1,n,KW Luxury Homes - lunch and cruise registration,47,8,3,3829206,,11,Keller Williams Northwest Region,7,Keller Williams Realty - c/o Mike Pfau,CHECK,"[{'address': '3840 East Ray Road', 'amount': 3...",8,8,1,"[{'availability': 1, 'cost': 25.0, 'event_id':...",932,1262197475,3,(Located on the north end of Lake Union),US,47.6511,-122.33,Waterways Cruises HomePort,WA
1,1366936200,2090,0,US,USD,1,"<p style=""font-family: 'Helvetica Neue', Helve...",rebuildingexchange.org,1362440224,1366504200,1.36244e+09,1366493400,0,765,0,0,1,y,WOOD BENCH - MAKE IT/ TAKE IT ONE DAY CLASS,43,9,59,5695572,<p>Our mission is to create a market for recla...,18,Rebuilding Exchange,10,,ACH,"[{'address': '', 'amount': 130.0, 'country': '...",47,47,1,"[{'availability': 1, 'cost': 79.76, 'event_id'...",507,1318631490,3,1740 W Webster Ave,US,41.9215,-87.6713,Rebuilding Exchange,IL
2,1356037200,4635,8,US,USD,1,"<span class=""vevent""><span class=""description""...",savings.com,1352056863,1355605200,0,1355594400,0,160,0,0,0,y,Colorado Grocery Savings and Couponing Workshop,47,23,53,4749437,,7,Savings Nation,8,,ACH,"[{'address': '', 'amount': 178.39, 'country': ...",None,41,1,"[{'availability': 1, 'cost': 13.92, 'event_id'...",250,1330452900,3,5955 South Holly Street,US,39.6089,-104.924,Koelbel Public Library,CO
3,1353801600,291,8,AU,AUD,1,<p>This workshop explores working with young p...,wodongatafe.edu.au,1351212497,1353369600,1.35121e+09,1353344400,0,366.57,0,None,1,y,Working with Young Parents and their Children ...,55,6,19,4658497,,0,Wodonga Institute of TAFE,0,,ACH,"[{'address': '', 'amount': 261.69, 'country': ...",25,25,1,"[{'availability': 1, 'cost': 40.37, 'event_id'...",158,1337565408,3,286 Napier St,AU,-36.7487,144.292,Lakeview Resort,VIC
4,1365969600,4803,8,US,USD,1,"<p style=""margin-bottom: 25px; color: rgb(59, ...",savings.com,1363374383,1365537600,0,1365534900,0,70,0,0,0,y,Suny Oswego Faculty Lunch and Learn Seminar,43,13,63,5853891,,7,Savings Nation,8,,ACH,"[{'address': '', 'amount': 178.39, 'country': ...",None,25,1,"[{'availability': 1, 'cost': 4.47, 'event_id':...",381,1330452900,3,7060 New York 104,US,43.4518,-76.5441,Suny Oswego - 201 Campus Center,NY


...

,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,event_end,event_published,event_start,fb_published,gts,has_analytics,has_header,has_logo,listed,name,name_length,num_order,num_payouts,object_id,org_desc,org_facebook,org_name,org_twitter,payee_name,payout_type,previous_payouts,sale_duration,sale_duration2,show_map,ticket_types,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state
0,1346626800,82,0,US,USD,0,<p>KW Luxury Homes Summit - Lunch and Cruise o...,kwspokane.com,1342719392,1346194800,1.34272e+09,1343413800,0,602.14,0,0,1,n,KW Luxury Homes - lunch and cruise registration,47,8,3,3829206,,11,Keller Williams Northwest Region,7,Keller Williams Realty - c/o Mike Pfau,CHECK,"[{'address': '3840 East Ray Road', 'amount': 3...",8,8,1,"[{'availability': 1, 'cost': 25.0, 'event_id':...",932,1262197475,3,(Located on the north end of Lake Union),US,47.6511,-122.33,Waterways Cruises HomePort,WA
1,1366936200,2090,0,US,USD,1,"<p style=""font-family: 'Helvetica Neue', Helve...",rebuildingexchange.org,1362440224,1366504200,1.36244e+09,1366493400,0,765,0,0,1,y,WOOD BENCH - MAKE IT/ TAKE IT ONE DAY CLASS,43,9,59,5695572,<p>Our mission is to create a market for recla...,18,Rebuilding Exchange,10,,ACH,"[{'address': '', 'amount': 130.0, 'country': '...",47,47,1,"[{'availability': 1, 'cost': 79.76, 'event_id'...",507,1318631490,3,1740 W Webster Ave,US,41.9215,-87.6713,Rebuilding Exchange,IL
2,1356037200,4635,8,US,USD,1,"<span class=""vevent""><span class=""description""...",savings.com,1352056863,1355605200,0,1355594400,0,160,0,0,0,y,Colorado Grocery Savings and Couponing Workshop,47,23,53,4749437,,7,Savings Nation,8,,ACH,"[{'address': '', 'amount': 178.39, 'country': ...",None,41,1,"[{'availability': 1, 'cost': 13.92, 'event_id'...",250,1330452900,3,5955 South Holly Street,US,39.6089,-104.924,Koelbel Public Library,CO
3,1353801600,291,8,AU,AUD,1,<p>This workshop explores working with young p...,wodongatafe.edu.au,1351212497,1353369600,1.35121e+09,1353344400,0,366.57,0,None,1,y,Working with Young Parents and their Children ...,55,6,19,4658497,,0,Wodonga Institute of TAFE,0,,ACH,"[{'address': '', 'amount': 261.69, 'country': ...",25,25,1,"[{'availability': 1, 'cost': 40.37, 'event_id'...",158,1337565408,3,286 Napier St,AU,-36.7487,144.292,Lakeview Resort,VIC
4,1365969600,4803,8,US,USD,1,"<p style=""margin-bottom: 25px; color: rgb(59, ...",savings.com,1363374383,1365537600,0,1365534900,0,70,0,0,0,y,Suny Oswego Faculty Lunch and Learn Seminar,43,13,63,5853891,,7,Savings Nation,8,,ACH,"[{'address': '', 'amount': 178.39, 'country': ...",None,25,1,"[{'availability': 1, 'cost': 4.47, 'event_id':...",381,1330452900,3,7060 New York 104,US,43.4518,-76.5441,Suny Oswego - 201 Campus Center,NY
5,1359613800,2082,4,GB,GBP,1,"<p style=""text-align: left;""><span style=""font...",youngs.co.uk,1358018059,1359181800,1.35802e+09,1359171000,1,102.76,0,None,1,y,Burns Night Dinner,18,1,0,5206710,"<p><br /><span style=""font-family: tahoma,aria...",30,Duke's Head Putney,15,,ACH,"[{'address': '', 'amount': 59.9, 'country': 'G...",13,13,1,"[{'availability': 1, 'cost': 29.95, 'event_id'...",0,1358015812,3,8 Lower Richmond Rd,GB,51.4672,-0.217113,Duke's Head Putney,Gt Lon
6,1353445200,2491,8,US,USD,1,"<p class=""MsoNormal"" style=""background: white;...",prsacentralohio.org,1350503498,1353013200,1.35093e+09,1353009600,0,405,0,0,1,y,November Luncheon - Social Media and the Ballo...,96,14,14,4620068,"<p> </p>\r\n<p class=""MsoNormal"">The <a href=""...",47,PRSA Central Ohio,15,,ACH,"[{'address': '', 'amount': 941.3, 'country': '...",24,29,1,"[{'availability': 1, 'cost': 22.63, 'event_id'...",469,1309961115,1,583 East Broad Street,US,39.9637,-82.9849,The Bluestone,OH
7,1328054400,2783,11,US,USD,1,<p>PHMSA's regulation (49 CFR Part 192) redefi...,aiworldwide.com,1319034449,1327622400,1.31904e+09,1327593600,0,1833.16,0,0,1,y,Pipeline Compliance in the Eagl

,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,event_end,event_published,event_start,fb_published,gts,has_analytics,has_header,has_logo,listed,name,name_length,num_order,num_payouts,object_id,org_desc,org_facebook,org_name,org_twitter,payee_name,payout_type,previous_payouts,sale_duration,sale_duration2,show_map,ticket_types,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state
0,1346626800,82,0,US,USD,0,<p>KW Luxury Homes Summit - Lunch and Cruise o...,kwspokane.com,1342719392,1346194800,1.34272e+09,1343413800,0,602.14,0,0,1,n,KW Luxury Homes - lunch and cruise registration,47,8,3,3829206,,11,Keller Williams Northwest Region,7,Keller Williams Realty - c/o Mike Pfau,CHECK,"[{'address': '3840 East Ray Road', 'amount': 3...",8,8,1,"[{'availability': 1, 'cost': 25.0, 'event_id':...",932,1262197475,3,(Located on the north end of Lake Union),US,47.6511,-122.33,Waterways Cruises HomePort,WA
1,1366936200,2090,0,US,USD,1,"<p style=""font-family: 'Helvetica Neue', Helve...",rebuildingexchange.org,1362440224,1366504200,1.36244e+09,1366493400,0,765,0,0,1,y,WOOD BENCH - MAKE IT/ TAKE IT ONE DAY CLASS,43,9,59,5695572,<p>Our mission is to create a market for recla...,18,Rebuilding Exchange,10,,ACH,"[{'address': '', 'amount': 130.0, 'country': '...",47,47,1,"[{'availability': 1, 'cost': 79.76, 'event_id'...",507,1318631490,3,1740 W Webster Ave,US,41.9215,-87.6713,Rebuilding Exchange,IL
2,1356037200,4635,8,US,USD,1,"<span class=""vevent""><span class=""description""...",savings.com,1352056863,1355605200,0,1355594400,0,160,0,0,0,y,Colorado Grocery Savings and Couponing Workshop,47,23,53,4749437,,7,Savings Nation,8,,ACH,"[{'address': '', 'amount': 178.39, 'country': ...",None,41,1,"[{'availability': 1, 'cost': 13.92, 'event_id'...",250,1330452900,3,5955 South Holly Street,US,39.6089,-104.924,Koelbel Public Library,CO
3,1353801600,291,8,AU,AUD,1,<p>This workshop explores working with young p...,wodongatafe.edu.au,1351212497,1353369600,1.35121e+09,1353344400,0,366.57,0,None,1,y,Working with Young Parents and their Children ...,55,6,19,4658497,,0,Wodonga Institute of TAFE,0,,ACH,"[{'address': '', 'amount': 261.69, 'country': ...",25,25,1,"[{'availability': 1, 'cost': 40.37, 'event_id'...",158,1337565408,3,286 Napier St,AU,-36.7487,144.292,Lakeview Resort,VIC
4,1365969600,4803,8,US,USD,1,"<p style=""margin-bottom: 25px; color: rgb(59, ...",savings.com,1363374383,1365537600,0,1365534900,0,70,0,0,0,y,Suny Oswego Faculty Lunch and Learn Seminar,43,13,63,5853891,,7,Savings Nation,8,,ACH,"[{'address': '', 'amount': 178.39, 'country': ...",None,25,1,"[{'availability': 1, 'cost': 4.47, 'event_id':...",381,1330452900,3,7060 New York 104,US,43.4518,-76.5441,Suny Oswego - 201 Campus Center,NY
5,1359613800,2082,4,GB,GBP,1,"<p style=""text-align: left;""><span style=""font...",youngs.co.uk,1358018059,1359181800,1.35802e+09,1359171000,1,102.76,0,None,1,y,Burns Night Dinner,18,1,0,5206710,"<p><br /><span style=""font-family: tahoma,aria...",30,Duke's Head Putney,15,,ACH,"[{'address': '', 'amount': 59.9, 'country': 'G...",13,13,1,"[{'availability': 1, 'cost': 29.95, 'event_id'...",0,1358015812,3,8 Lower Richmond Rd,GB,51.4672,-0.217113,Duke's Head Putney,Gt Lon
6,1353445200,2491,8,US,USD,1,"<p class=""MsoNormal"" style=""background: white;...",prsacentralohio.org,1350503498,1353013200,1.35093e+09,1353009600,0,405,0,0,1,y,November Luncheon - Social Media and the Ballo...,96,14,14,4620068,"<p> </p>\r\n<p class=""MsoNormal"">The <a href=""...",47,PRSA Central Ohio,15,,ACH,"[{'address': '', 'amount': 941.3, 'country': '...",24,29,1,"[{'availability': 1, 'cost': 22.63, 'event_id'...",469,1309961115,1,583 East Broad Street,US,39.9637,-82.9849,The Bluestone,OH
7,1328054400,2783,11,US,USD,1,<p>PHMSA's regulation (49 CFR Part 192) redefi...,aiworldwide.com,1319034449,1327622400,1.31904e+09,1327593600,0,1833.16,0,0,1,y,Pipeline Compliance in the Eagl

,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,event_end,event_published,event_start,fb_published,gts,has_analytics,has_header,has_logo,listed,name,name_length,num_order,num_payouts,object_id,org_desc,org_facebook,org_name,org_twitter,payee_name,payout_type,previous_payouts,sale_duration,sale_duration2,show_map,ticket_types,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state
0,1346626800,82,0,US,USD,0,<p>KW Luxury Homes Summit - Lunch and Cruise o...,kwspokane.com,1342719392,1346194800,1.34272e+09,1343413800,0,602.14,0,0,1,n,KW Luxury Homes - lunch and cruise registration,47,8,3,3829206,,11,Keller Williams Northwest Region,7,Keller Williams Realty - c/o Mike Pfau,CHECK,"[{'address': '3840 East Ray Road', 'amount': 3...",8,8,1,"[{'availability': 1, 'cost': 25.0, 'event_id':...",932,1262197475,3,(Located on the north end of Lake Union),US,47.6511,-122.33,Waterways Cruises HomePort,WA
1,1366936200,2090,0,US,USD,1,"<p style=""font-family: 'Helvetica Neue', Helve...",rebuildingexchange.org,1362440224,1366504200,1.36244e+09,1366493400,0,765,0,0,1,y,WOOD BENCH - MAKE IT/ TAKE IT ONE DAY CLASS,43,9,59,5695572,<p>Our mission is to create a market for recla...,18,Rebuilding Exchange,10,,ACH,"[{'address': '', 'amount': 130.0, 'country': '...",47,47,1,"[{'availability': 1, 'cost': 79.76, 'event_id'...",507,1318631490,3,1740 W Webster Ave,US,41.9215,-87.6713,Rebuilding Exchange,IL
2,1356037200,4635,8,US,USD,1,"<span class=""vevent""><span class=""description""...",savings.com,1352056863,1355605200,0,1355594400,0,160,0,0,0,y,Colorado Grocery Savings and Couponing Workshop,47,23,53,4749437,,7,Savings Nation,8,,ACH,"[{'address': '', 'amount': 178.39, 'country': ...",None,41,1,"[{'availability': 1, 'cost': 13.92, 'event_id'...",250,1330452900,3,5955 South Holly Street,US,39.6089,-104.924,Koelbel Public Library,CO
3,1353801600,291,8,AU,AUD,1,<p>This workshop explores working with young p...,wodongatafe.edu.au,1351212497,1353369600,1.35121e+09,1353344400,0,366.57,0,None,1,y,Working with Young Parents and their Children ...,55,6,19,4658497,,0,Wodonga Institute of TAFE,0,,ACH,"[{'address': '', 'amount': 261.69, 'country': ...",25,25,1,"[{'availability': 1, 'cost': 40.37, 'event_id'...",158,1337565408,3,286 Napier St,AU,-36.7487,144.292,Lakeview Resort,VIC
4,1365969600,4803,8,US,USD,1,"<p style=""margin-bottom: 25px; color: rgb(59, ...",savings.com,1363374383,1365537600,0,1365534900,0,70,0,0,0,y,Suny Oswego Faculty Lunch and Learn Seminar,43,13,63,5853891,,7,Savings Nation,8,,ACH,"[{'address': '', 'amount': 178.39, 'country': ...",None,25,1,"[{'availability': 1, 'cost': 4.47, 'event_id':...",381,1330452900,3,7060 New York 104,US,43.4518,-76.5441,Suny Oswego - 201 Campus Center,NY
5,1359613800,2082,4,GB,GBP,1,"<p style=""text-align: left;""><span style=""font...",youngs.co.uk,1358018059,1359181800,1.35802e+09,1359171000,1,102.76,0,None,1,y,Burns Night Dinner,18,1,0,5206710,"<p><br /><span style=""font-family: tahoma,aria...",30,Duke's Head Putney,15,,ACH,"[{'address': '', 'amount': 59.9, 'country': 'G...",13,13,1,"[{'availability': 1, 'cost': 29.95, 'event_id'...",0,1358015812,3,8 Lower Richmond Rd,GB,51.4672,-0.217113,Duke's Head Putney,Gt Lon
6,1353445200,2491,8,US,USD,1,"<p class=""MsoNormal"" style=""background: white;...",prsacentralohio.org,1350503498,1353013200,1.35093e+09,1353009600,0,405,0,0,1,y,November Luncheon - Social Media and the Ballo...,96,14,14,4620068,"<p> </p>\r\n<p class=""MsoNormal"">The <a href=""...",47,PRSA Central Ohio,15,,ACH,"[{'address': '', 'amount': 941.3, 'country': '...",24,29,1,"[{'availability': 1, 'cost': 22.63, 'event_id'...",469,1309961115,1,583 East Broad Street,US,39.9637,-82.9849,The Bluestone,OH
7,1328054400,2783,11,US,USD,1,<p>PHMSA's regulation (49 CFR Part 192) redefi...,aiworldwide.com,1319034449,1327622400,1.31904e+09,1327593600,0,1833.16,0,0,1,y,Pipeline Compliance in the Eagl

,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,event_end,event_published,event_start,fb_published,gts,has_analytics,has_header,has_logo,listed,name,name_length,num_order,num_payouts,object_id,org_desc,org_facebook,org_name,org_twitter,payee_name,payout_type,previous_payouts,sale_duration,sale_duration2,show_map,ticket_types,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state
0,1346626800,82,0,US,USD,0,<p>KW Luxury Homes Summit - Lunch and Cruise o...,kwspokane.com,1342719392,1346194800,1.34272e+09,1343413800,0,602.14,0,0,1,n,KW Luxury Homes - lunch and cruise registration,47,8,3,3829206,,11,Keller Williams Northwest Region,7,Keller Williams Realty - c/o Mike Pfau,CHECK,"[{'address': '3840 East Ray Road', 'amount': 3...",8,8,1,"[{'availability': 1, 'cost': 25.0, 'event_id':...",932,1262197475,3,(Located on the north end of Lake Union),US,47.6511,-122.33,Waterways Cruises HomePort,WA
1,1366936200,2090,0,US,USD,1,"<p style=""font-family: 'Helvetica Neue', Helve...",rebuildingexchange.org,1362440224,1366504200,1.36244e+09,1366493400,0,765,0,0,1,y,WOOD BENCH - MAKE IT/ TAKE IT ONE DAY CLASS,43,9,59,5695572,<p>Our mission is to create a market for recla...,18,Rebuilding Exchange,10,,ACH,"[{'address': '', 'amount': 130.0, 'country': '...",47,47,1,"[{'availability': 1, 'cost': 79.76, 'event_id'...",507,1318631490,3,1740 W Webster Ave,US,41.9215,-87.6713,Rebuilding Exchange,IL
2,1356037200,4635,8,US,USD,1,"<span class=""vevent""><span class=""description""...",savings.com,1352056863,1355605200,0,1355594400,0,160,0,0,0,y,Colorado Grocery Savings and Couponing Workshop,47,23,53,4749437,,7,Savings Nation,8,,ACH,"[{'address': '', 'amount': 178.39, 'country': ...",None,41,1,"[{'availability': 1, 'cost': 13.92, 'event_id'...",250,1330452900,3,5955 South Holly Street,US,39.6089,-104.924,Koelbel Public Library,CO
3,1353801600,291,8,AU,AUD,1,<p>This workshop explores working with young p...,wodongatafe.edu.au,1351212497,1353369600,1.35121e+09,1353344400,0,366.57,0,None,1,y,Working with Young Parents and their Children ...,55,6,19,4658497,,0,Wodonga Institute of TAFE,0,,ACH,"[{'address': '', 'amount': 261.69, 'country': ...",25,25,1,"[{'availability': 1, 'cost': 40.37, 'event_id'...",158,1337565408,3,286 Napier St,AU,-36.7487,144.292,Lakeview Resort,VIC
4,1365969600,4803,8,US,USD,1,"<p style=""margin-bottom: 25px; color: rgb(59, ...",savings.com,1363374383,1365537600,0,1365534900,0,70,0,0,0,y,Suny Oswego Faculty Lunch and Learn Seminar,43,13,63,5853891,,7,Savings Nation,8,,ACH,"[{'address': '', 'amount': 178.39, 'country': ...",None,25,1,"[{'availability': 1, 'cost': 4.47, 'event_id':...",381,1330452900,3,7060 New York 104,US,43.4518,-76.5441,Suny Oswego - 201 Campus Center,NY
5,1359613800,2082,4,GB,GBP,1,"<p style=""text-align: left;""><span style=""font...",youngs.co.uk,1358018059,1359181800,1.35802e+09,1359171000,1,102.76,0,None,1,y,Burns Night Dinner,18,1,0,5206710,"<p><br /><span style=""font-family: tahoma,aria...",30,Duke's Head Putney,15,,ACH,"[{'address': '', 'amount': 59.9, 'country': 'G...",13,13,1,"[{'availability': 1, 'cost': 29.95, 'event_id'...",0,1358015812,3,8 Lower Richmond Rd,GB,51.4672,-0.217113,Duke's Head Putney,Gt Lon
6,1353445200,2491,8,US,USD,1,"<p class=""MsoNormal"" style=""background: white;...",prsacentralohio.org,1350503498,1353013200,1.35093e+09,1353009600,0,405,0,0,1,y,November Luncheon - Social Media and the Ballo...,96,14,14,4620068,"<p> </p>\r\n<p class=""MsoNormal"">The <a href=""...",47,PRSA Central Ohio,15,,ACH,"[{'address': '', 'amount': 941.3, 'country': '...",24,29,1,"[{'availability': 1, 'cost': 22.63, 'event_id'...",469,1309961115,1,583 East Broad Street,US,39.9637,-82.9849,The Bluestone,OH
7,1328054400,2783,11,US,USD,1,<p>PHMSA's regulation (49 CFR Part 192) redefi...,aiworldwide.com,1319034449,1327622400,1.31904e+09,1327593600,0,1833.16,0,0,1,y,Pipeline Compliance in the Eagl

KeyboardInterrupt: 

In [166]:
df

,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,event_end,event_published,event_start,fb_published,gts,has_analytics,has_header,has_logo,listed,name,name_length,num_order,num_payouts,object_id,org_desc,org_facebook,org_name,org_twitter,payee_name,payout_type,previous_payouts,sale_duration,sale_duration2,show_map,ticket_types,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state
0,1346626800,82,0,US,USD,0,<p>KW Luxury Homes Summit - Lunch and Cruise o...,kwspokane.com,1342719392,1346194800,1.34272e+09,1343413800,0,602.14,0,0,1,n,KW Luxury Homes - lunch and cruise registration,47,8,3,3829206,,11,Keller Williams Northwest Region,7,Keller Williams Realty - c/o Mike Pfau,CHECK,"[{'address': '3840 East Ray Road', 'amount': 3...",8,8,1,"[{'availability': 1, 'cost': 25.0, 'event_id':...",932,1262197475,3,(Located on the north end of Lake Union),US,47.6511,-122.33,Waterways Cruises HomePort,WA
1,1366936200,2090,0,US,USD,1,"<p style=""font-family: 'Helvetica Neue', Helve...",rebuildingexchange.org,1362440224,1366504200,1.36244e+09,1366493400,0,765,0,0,1,y,WOOD BENCH - MAKE IT/ TAKE IT ONE DAY CLASS,43,9,59,5695572,<p>Our mission is to create a market for recla...,18,Rebuilding Exchange,10,,ACH,"[{'address': '', 'amount': 130.0, 'country': '...",47,47,1,"[{'availability': 1, 'cost': 79.76, 'event_id'...",507,1318631490,3,1740 W Webster Ave,US,41.9215,-87.6713,Rebuilding Exchange,IL
2,1356037200,4635,8,US,USD,1,"<span class=""vevent""><span class=""description""...",savings.com,1352056863,1355605200,0,1355594400,0,160,0,0,0,y,Colorado Grocery Savings and Couponing Workshop,47,23,53,4749437,,7,Savings Nation,8,,ACH,"[{'address': '', 'amount': 178.39, 'country': ...",None,41,1,"[{'availability': 1, 'cost': 13.92, 'event_id'...",250,1330452900,3,5955 South Holly Street,US,39.6089,-104.924,Koelbel Public Library,CO
3,1353801600,291,8,AU,AUD,1,<p>This workshop explores working with young p...,wodongatafe.edu.au,1351212497,1353369600,1.35121e+09,1353344400,0,366.57,0,None,1,y,Working with Young Parents and their Children ...,55,6,19,4658497,,0,Wodonga Institute of TAFE,0,,ACH,"[{'address': '', 'amount': 261.69, 'country': ...",25,25,1,"[{'availability': 1, 'cost': 40.37, 'event_id'...",158,1337565408,3,286 Napier St,AU,-36.7487,144.292,Lakeview Resort,VIC
4,1365969600,4803,8,US,USD,1,"<p style=""margin-bottom: 25px; color: rgb(59, ...",savings.com,1363374383,1365537600,0,1365534900,0,70,0,0,0,y,Suny Oswego Faculty Lunch and Learn Seminar,43,13,63,5853891,,7,Savings Nation,8,,ACH,"[{'address': '', 'amount': 178.39, 'country': ...",None,25,1,"[{'availability': 1, 'cost': 4.47, 'event_id':...",381,1330452900,3,7060 New York 104,US,43.4518,-76.5441,Suny Oswego - 201 Campus Center,NY
5,1359613800,2082,4,GB,GBP,1,"<p style=""text-align: left;""><span style=""font...",youngs.co.uk,1358018059,1359181800,1.35802e+09,1359171000,1,102.76,0,None,1,y,Burns Night Dinner,18,1,0,5206710,"<p><br /><span style=""font-family: tahoma,aria...",30,Duke's Head Putney,15,,ACH,"[{'address': '', 'amount': 59.9, 'country': 'G...",13,13,1,"[{'availability': 1, 'cost': 29.95, 'event_id'...",0,1358015812,3,8 Lower Richmond Rd,GB,51.4672,-0.217113,Duke's Head Putney,Gt Lon
6,1353445200,2491,8,US,USD,1,"<p class=""MsoNormal"" style=""background: white;...",prsacentralohio.org,1350503498,1353013200,1.35093e+09,1353009600,0,405,0,0,1,y,November Luncheon - Social Media and the Ballo...,96,14,14,4620068,"<p> </p>\r\n<p class=""MsoNormal"">The <a href=""...",47,PRSA Central Ohio,15,,ACH,"[{'address': '', 'amount': 941.3, 'country': '...",24,29,1,"[{'availability': 1, 'cost': 22.63, 'event_id'...",469,1309961115,1,583 East Broad Street,US,39.9637,-82.9849,The Bluestone,OH
7,1328054400,2783,11,US,USD,1,<p>PHMSA's regulation (49 CFR Part 192) redefi...,aiworldwide.com,1319034449,1327622400,1.31904e+09,1327593600,0,1833.16,0,0,1,y,Pipeline Compliance in the Eagl

In [16]:
data = pd.read_json('/Users/josh-mantovani/Desktop/fraud_prediction/data/data.json')

In [385]:
data.head()

,acct_type,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,event_end,event_published,event_start,fb_published,gts,has_analytics,has_header,has_logo,listed,name,name_length,num_order,num_payouts,object_id,org_desc,org_facebook,org_name,org_twitter,payee_name,payout_type,previous_payouts,sale_duration,sale_duration2,show_map,ticket_types,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state
0,fraudster_event,1266062400,3852,5,US,USD,0.0,"<p><a href=""http://s432.photobucket.com/albums...",gmail.com,1262739706,1265630400,1.263110e+09,1265594400,0,0.00,0,1.0,0,y,"99 HOUR ""NO SLEEP"" SUPER BOWL CELEBRITY WEEKEN...",60,0,0,527017,,0.0,Party Starz Ent & Diverse Int'l Group,0.0,,,[],29.0,33,1,"[{'event_id': 527017, 'cost': 25.0, 'availabil...",36,1259613950,1,717 Washington Avenue,US,25.777471,-80.133433,INK Nightclub - South Beach,FL
1,premium,1296720000,3499,0,US,USD,1.0,"<p>Join us for a quick, one-night, community-b...",ruf.org,1293832670,1296288000,1.293833e+09,1296255600,0,868.02,0,0.0,1,n,Winthrop RUF Winter Getaway,27,23,1,786878,"<p>Since 1987, RUF has ministered to students ...",0.0,RUF at Winthrop University,12.0,RUF,CHECK,"[{'name': 'RUF', 'created': '2010-10-01 01:10:...",28.0,28,0,"[{'event_id': 786878, 'cost': 35.0, 'availabil...",149,1280942776,3,,US,32.776566,-79.930922,"The Charleston, SC area",SC
2,premium,1296172800,2601,8,US,USD,1.0,"<h3><span class=""subcategory""><strong>Teacher ...",pvsd.k12.ca.us,1291090956,1295740800,1.291092e+09,1295713800,0,3500.00,0,NaN,0,y,DRDP (2010) Teacher Training,28,19,4,787337,"<p><a href=""http://www.desiredresults.com"">www...",0.0,University Preparation School in collaboration...,0.0,University Preparation School,CHECK,"[{'name': 'Danielle Severn', 'created': '2010-...",54.0,54,0,"[{'event_id': 787337, 'cost': 93.51, 'availabi...",214,1272559388,3,10100 Pioneer Blvd Suite 100,US,33.944201,-118.080419,Los Angeles County Office of Education,CA
3,premium,1388966400,12347,6,IE,EUR,1.0,"<p style=""margin-bottom: 1.3em; padding-bottom...",irishtabletennis.com,1360681570,1388534400,1.360683e+09,1360702800,0,1167.35,0,0.0,1,y,ITTA Affiliation 2013,21,39,16,885645,,0.0,,0.0,,ACH,"[{'name': '', 'created': '2010-11-09 01:10:15'...",0.0,0,0,"[{'event_id': 885645, 'cost': 25.0, 'availabil...",889,1283870102,3,,None,NaN,NaN,None,None
4,premium,1297900800,2417,11,US,USD,0.0,<p>Writers and filmmakers need to understand t...,artsandbusinesscouncil.org,1291994666,1297468800,1.291995e+09,1297440000,1,2313.15,0,0.0,0,y,Self Defense for Writers and Filmmakers: For A...,66,30,0,1114349,"<p><font face=""Arial""><font size=""2""><strong>T...",0.0,Arts & Business Council of Greater Boston,0.0,Arts and Business Council or Greater Boston,CHECK,[{'name': 'Arts and Business Council or Greate...,63.0,63,0,"[{'event_id': 1114349, 'cost': 150.0, 'availab...",35,1288984065,3,One Marina Park Drive,US,42.353848,-71.044276,Fish & Richardson,MA


In [17]:
y = data['acct_type'].str.contains('fraud')
X = data.drop(['acct_type'], axis=1)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    test_size=0.20, 
                                                    stratify=y)

In [25]:
def tokenize_and_lemmatize(text):
    text = str(text).lower()
    for sep in seperators:
        text = text.replace(sep, ' ')
    for html in htmls:
        text = text.replace(html, '')
    for space in spaces:
        text = text.replace(space, ' ')
    doc = nlp(text)
    words = []
    for token in doc:
        lemma = token.lemma_
        if not token.is_stop and not token.is_punct and lemma not in my_stops:
            words.append(lemma)
    return words

nlp_pipe = Pipeline(steps=[
    ('vec', CountVectorizer(ngram_range=(1,2),
                            max_df=0.9,
                            max_features=10000,
                            token_pattern=None,
                            tokenizer=tokenize_and_lemmatize)),
    ('bayes', MultinomialNB())
])
nlp_pipe

Pipeline(steps=[('vec',
                 CountVectorizer(max_df=0.9, max_features=10000,
                                 ngram_range=(1, 2), token_pattern=None,
                                 tokenizer=<function tokenize_and_lemmatize at 0x7fdb7e32ea70>)),
                ('bayes', MultinomialNB())])

In [452]:
nlp_pipe.fit(X_train['description'], y_train)

Pipeline(steps=[('vec',
                 CountVectorizer(max_df=0.9, max_features=10000,
                                 ngram_range=(1, 2), token_pattern=None,
                                 tokenizer=<function tokenize_and_lemmatize at 0x7fd4d1f25290>)),
                ('bayes', MultinomialNB())])

In [558]:
nlp_pipe.predict_proba(X_train['description'])

array([[3.01666992e-08, 9.99999970e-01],
       [9.93314648e-01, 6.68535169e-03],
       [5.70415798e-02, 9.42958420e-01],
       ...,
       [3.68747854e-01, 6.31252146e-01],
       [9.99907335e-01, 9.26654452e-05],
       [9.99881815e-01, 1.18185268e-04]])

In [454]:
grid = {"vec__ngram_range": [(1, 1), (1, 2), (1, 3), (2, 2)],
       "vec__max_df": [0.90, 0.95, 0.85, 0.80],
       "vec__max_features": [10000, 100000, 80000]}

search = GridSearchCV(nlp_pipe, param_grid=grid, scoring='f1', n_jobs=-1, cv=5)
search.fit(X_train['description'], y_train)
search.best_estimator_

Pipeline(steps=[('vec',
                 CountVectorizer(max_df=0.9, max_features=100000,
                                 ngram_range=(1, 2), token_pattern=None,
                                 tokenizer=<function tokenize_and_lemmatize at 0x7fd4d1f25290>)),
                ('bayes', MultinomialNB())])

In [456]:
search.best_score_

0.3501102220469649

In [457]:
search.best_params_

{'vec__max_df': 0.9, 'vec__max_features': 100000, 'vec__ngram_range': (1, 2)}

In [455]:
best_nlp_pipe = search.best_estimator_
joblib.dump(best_nlp_pipe, '../../models/nlp_pipe.joblib')

['../../models/nlp_pipe.joblib']

In [19]:
def engineer_features(_df):
    """Engineer several features based on EDA that may be useful in prediction including
    transforming the string descriptions to a probability of fraud"""
    df = _df.copy()
    #Several columns need to become datetimes in order to perform the necessary operations
    cols_to_dt = [
    'event_created', 'event_end', 'event_start', 'user_created', 'approx_payout_date'
    ]
    for col in cols_to_dt:
        df[col] = df[col].astype(int).apply(dt.datetime.utcfromtimestamp)
    #setting up popular countries list for future use
    popular_countries = ['US', 'GB', 'AU']

    #Defining several functions that will be used below to transform the data
    def transform_description_to_proba(descriptions):
        "Takes NLP pipe trained on training data only to transform descriptions to prob of fraud"
        nlp_pipe = joblib.load('../../models/nlp_pipe.joblib')
        return nlp_pipe.predict_proba(descriptions)[:, 1]

    def get_ticket_price(val):
        """Calculates average ticket price across all training events"""
        price = []
        for ticket in val:
            price.append(ticket['cost'])
        if len(price):
            return np.mean(price)
        else:
            return 0

    def get_ticket_available(val):
        """Calculates number of tickets available"""
        available = []
        for ticket in val:
            available.append(ticket['quantity_total'])
        if sum(available):
            return sum(available)
        else:
            return 1

    def get_avg_quantity_sold(val):
        "Gets avg number of tickets sold across all ticket types"
        num_sold = 0
        for ticket in val:
            num_sold += ticket['quantity_sold']
        return np.mean(num_sold)

    def get_num_payouts(val):
        "Gets number of previous payouts"
        number_of_payouts = len(val)
        return number_of_payouts

    def payee_name_in_org_name(row):
        "Calculating a bool feature to represent the payee name being in the org name"
        return row['payee_name'] in row['org_name']

    def venue_country_is_source_country(row):
        "Checking if the venue country is the country the event was created in"
        return row['venue_country']==row['country']
    
    #Using the previously defined functions to transform the data
    df['avg_ticket_price'] = df['ticket_types'].apply(get_ticket_price)
    df['total_tickets_available'] = df['ticket_types'].apply(get_ticket_available)
    df['number_of_payouts'] = df['previous_payouts'].apply(get_num_payouts)
    df['avg_quantity_sold'] = df['ticket_types'].apply(get_avg_quantity_sold)
    
    #Combining multiple columns:
    df['payee_name_in_org_name'] = df.apply(payee_name_in_org_name, axis=1)
    df['venue_country_is_source_country'] = df.apply(venue_country_is_source_country, axis=1)
    df['avg_cost_per_ticket'] = df['avg_ticket_price'] / df['total_tickets_available']
 
    
    #Transforming description:
    df['description'] = transform_description_to_proba(df['description'])
    
    #Datetime calculations:
    df['days_to_event'] = (df['event_start'] - df['event_created']).dt.days
    df['event_length'] = (df['event_end'] - df['event_start']).dt.days
    
    df['pop_country'] = df['country'].isin(popular_countries)

    #The following columns have already had information extracted from them, were deemed not useful through EDA,
    #Or I could not tell what the column represented and thus it may introduce data leakage
    cols_to_del = ['gts', 'email_domain', 'num_order',
                   'object_id', 'org_facebook', 'org_twitter',
                   'previous_payouts', 'payee_name', 'org_name',
                   'venue_latitude', 'venue_longitude', 'venue_name',
                   'event_published', 'channels', 'sale_duration2',
                    'user_type', 'ticket_types', 'venue_address',
                  'sale_duration', 'channels', 'approx_payout_date',
                  'name', 'org_desc']
    cols_to_del = cols_to_del + cols_to_dt

    df.drop(cols_to_del, axis=1, inplace=True)
    df.replace(to_replace='', value=np.nan, inplace=True)
    
    #changing dtypes:
    num_cols = [
    'body_length', 'name_length', 'num_payouts', 'user_age', 'days_to_event',
    'event_length', 'avg_ticket_price', 'total_tickets_available',
    'avg_cost_per_ticket', 'number_of_payouts', 'avg_quantity_sold',
    'description'
    ]
    
    cat_cols = [col for col in df.columns if col not in num_cols]
    df[cat_cols] = df[cat_cols].astype('category')
    df[num_cols] = df[num_cols].astype(float)
    
    return df

In [32]:
engineer_features(X_train).info()

KeyboardInterrupt: 

In [33]:
eng_feats = preprocessing.FunctionTransformer(engineer_features)
eng_feats

FunctionTransformer(func=<function engineer_features at 0x7fdb31d06320>)

In [34]:
X_train_processed = eng_feats.transform(X_train)
# nlp_pipe.fit(X_train_processed['description'], y_train)
# X_train_processed['description'] = nlp_pipe.predict_proba(X_train_processed['description'])
X_train_processed


,body_length,country,currency,delivery_method,description,fb_published,has_analytics,has_header,has_logo,listed,...,avg_ticket_price,total_tickets_available,number_of_payouts,avg_quantity_sold,payee_name_in_org_name,venue_country_is_source_country,avg_cost_per_ticket,days_to_event,event_length,pop_country
7270,8277.0,BG,GBP,0.0,1.000000e+00,0,0,NaN,1,y,...,85.000,200.0,0.0,0.0,True,False,0.425000,5.0,0.0,False
9877,3160.0,US,USD,0.0,2.097659e-44,0,0,NaN,1,y,...,15.545,35.0,2.0,3.0,True,True,0.444143,24.0,0.0,True
1539,1797.0,GB,GBP,1.0,7.029726e-11,0,0,1.0,1,n,...,6.000,1900.0,42.0,950.0,True,False,0.003158,23.0,0.0,True
9032,6105.0,US,USD,0.0,1.860558e-118,1,0,0.0,1,y,...,13.750,441.0,8.0,168.0,True,True,0.031179,45.0,1.0,True
2794,631.0,AU,AUD,1.0,6.246124e-29,0,0,NaN,1,y,...,81.730,25.0,68.0,9.0,True,True,3.269200,111.0,1.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671,11272.0,US,USD,1.0,0.000000e+00,0,0,0.0,1,y,...,87.000,150.0,12.0,14.0,True,True,0.580000,125.0,0.0,True
4876,1246.0,AU,AUD,1.0,5.276418e-11,0,0,0.0,1,y,...,49.000,100.0,28.0,1.0,True,False,0.490000,20.0,0.0,True
1881,1794.0,US,USD,0.0,8.580957e-53,0,0,NaN,1,y,...,20.000,50.0,4.0,41.0,False,True,0.400000,40.0,0.0,True
12396,7429.0,GB,GBP,0.0,3.518453e-168,0,0,0.0,1,y,...,282.995,24.0,7.0,7.0,True,False,11.791458,134.0,0.0,True


In [518]:
# num_cols = [
#     'body_length', 'name_length', 'num_payouts', 'user_age', 'days_to_event',
#     'event_length', 'avg_ticket_price', 'total_tickets_available',
#     'avg_cost_per_ticket', 'number_of_payouts', 'avg_quantity_sold',
#     'description'
# ]
# cat_cols = [col for col in X_train_processed.columns if col not in num_cols]
# X_train_processed[cat_cols] = X_train_processed[cat_cols].astype('category')
# X_train_processed.info()
print(list(X_train_processed.select_dtypes(include='category').columns))

['country', 'currency', 'delivery_method', 'fb_published', 'has_analytics', 'has_header', 'has_logo', 'listed', 'payout_type', 'show_map', 'venue_country', 'venue_state', 'payee_name_in_org_name', 'venue_country_is_source_country', 'pop_country']


In [484]:
# trans = ColumnTransformer(transformers=[
#     ('nlp_pipe', nlp_pipe, ['description'])
# ], remainder='passthrough')

# preprocess_data = Pipeline(steps=[
#     ('fe', eng_feats),
#     ('bayes', trans)
# ])
# preprocess_data.fit(X_train)

In [485]:
def process_description(d, col_name='description'):
    pass

In [583]:
num_cols = ['body_length', 'description', 'name_length', 'num_payouts', 'user_age', 
            'avg_ticket_price', 'total_tickets_available', 'number_of_payouts', 
            'avg_quantity_sold', 'avg_cost_per_ticket', 'days_to_event', 'event_length']

cat_cols = ['country', 'currency', 'delivery_method', 'fb_published', 'has_analytics', 
            'has_header', 'has_logo', 'listed', 'payout_type', 'show_map', 'venue_country', 
            'venue_state', 'payee_name_in_org_name', 'venue_country_is_source_country', 'pop_country']

num_transformer = Pipeline(steps=[
    ('imputer', impute.KNNImputer(add_indicator=True)),
    ('std', preprocessing.StandardScaler())
])

cat_transformer = Pipeline(steps=[
    ('imputer', impute.SimpleImputer(strategy='most_frequent', add_indicator=True)),
    ('onehot', preprocessing.OneHotEncoder(handle_unknown='ignore'))
])

trans = ColumnTransformer(transformers=[
    ('numeric_transformer', num_transformer, num_cols),
    ('categorical_transformer', cat_transformer, cat_cols)
])

pipe = smote_pipeline(steps=[
    ('fe', eng_feats),
    ('transformer', trans),
    ('smoter', SMOTE(n_jobs=-1)),
    ('cls', RandomForestClassifier(n_jobs=-1, n_estimators=1000))
])
pipe

Pipeline(steps=[('fe',
                 FunctionTransformer(func=<function engineer_features at 0x7fd4ab29db90>)),
                ('transformer',
                 ColumnTransformer(transformers=[('numeric_transformer',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(add_indicator=True)),
                                                                  ('std',
                                                                   StandardScaler())]),
                                                  ['body_length', 'description',
                                                   'name_length', 'num_payouts',
                                                   'user_age',
                                                   'avg_ticket_price',
                                                   'total_tickets...
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['country', 'currency',
                                                   'delivery_method',
                                                   'fb_published',
                                                   'has_analytics',
                                                   'has_header', 'has_logo',
                                                   'listed', 'payout_type',
                                                   'show_map', 'venue_country',
                                                   'venue_state',
                                                   'payee_name_in_org_name',
                                                   'venue_country_is_source_country',
                                                   'pop_country'])])),
                ('smoter', SMOTE(n_jobs=-1)),
                ('cls', RandomForestClassifier(n_estimators=1000, n_jobs=-1))])

In [586]:
joblib.dump(pipe, '../../models/prediction_pipeline.joblib')

['../../models/prediction_pipeline.joblib']

In [584]:
pipe.fit(X_train,y_train)

Pipeline(steps=[('fe',
                 FunctionTransformer(func=<function engineer_features at 0x7fd4ab29db90>)),
                ('transformer',
                 ColumnTransformer(transformers=[('numeric_transformer',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(add_indicator=True)),
                                                                  ('std',
                                                                   StandardScaler())]),
                                                  ['body_length', 'description',
                                                   'name_length', 'num_payouts',
                                                   'user_age',
                                                   'avg_ticket_price',
                                                   'total_tickets...
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['country', 'currency',
                                                   'delivery_method',
                                                   'fb_published',
                                                   'has_analytics',
                                                   'has_header', 'has_logo',
                                                   'listed', 'payout_type',
                                                   'show_map', 'venue_country',
                                                   'venue_state',
                                                   'payee_name_in_org_name',
                                                   'venue_country_is_source_country',
                                                   'pop_country'])])),
                ('smoter', SMOTE(n_jobs=-1)),
                ('cls', RandomForestClassifier(n_estimators=1000, n_jobs=-1))])

In [585]:
print(f1_score(y_train, pipe.predict(X_train)))

print(f1_score(y_test, pipe.predict(X_test)))

1.0
0.935672514619883


In [564]:
scores = cross_val_score(pipe, X_train, y_train, cv=3, scoring='f1', n_jobs=-1, error_score='raise')
print(f'Average score: {np.mean(scores)}\nScores: {scores}')

AttributeError: module '__main__' has no attribute 'tokenize_and_lemmatize'

In [534]:
test_predictions = pipe.predict(X_test)
f1_score(y_test, test_predictions)

0.9291338582677164

In [503]:
len(test_predictions)==len(y_test)

True

In [482]:
for col in cat_cols:
    print(col)
    impute.SimpleImputer(strategy='most_frequent').fit_transform(X_train_processed[[col]].astype('category'))


country
currency
delivery_method
fb_published
has_analytics
has_header
has_logo
listed
payout_type
show_map
venue_country
venue_state
payee_name_in_org_name
venue_country_is_source_country
pop_country


In [477]:
X_train_processed['country'].unique()

array(['US', 'GB', 'CA', 'QA', 'VI', 'IE', 'AU', nan, 'NA', 'DE', 'BE',
       'EC', 'AR', 'ES', 'ZA', 'DZ', None, 'NZ', 'CH', 'HR', 'VN', 'RS',
       'FR', 'PH', 'IL', 'HU', 'NL', 'PR', 'TR', 'DK', 'NG', 'ID', 'RO',
       'MA', 'PK', 'IM', 'CZ', 'GR', 'MY', 'GH', 'JM', 'A1', 'TH', 'MX',
       'CI', 'IN', 'PS', 'KH', 'AT', 'SE', 'CN', 'FI', 'KE', 'LB', 'CR',
       'SG', 'JE', 'VE', 'IS', 'CO', 'IT', 'RU', 'NI', 'BG', 'CM', 'AE',
       'UY', 'BS', 'SI'], dtype=object)

In [543]:
test = joblib.load('../../models/nlp_pipe.joblib')
test

Pipeline(steps=[('vec',
                 CountVectorizer(max_df=0.9, max_features=100000,
                                 ngram_range=(1, 2), token_pattern=None,
                                 tokenizer=<function tokenize_and_lemmatize at 0x7fd4d1f25290>)),
                ('bayes', MultinomialNB())])

In [544]:
test = joblib.load('../../models/prediction_pipeline.joblib')
test

Pipeline(steps=[('fe',
                 FunctionTransformer(func=<function engineer_features at 0x7fd4797bcc20>)),
                ('transformer',
                 ColumnTransformer(transformers=[('numeric_transformer',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(add_indicator=True)),
                                                                  ('std',
                                                                   StandardScaler())]),
                                                  ['body_length', 'description',
                                                   'name_length', 'num_payouts',
                                                   'user_age',
                                                   'avg_ticket_price',
                                                   'total_tickets...
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['country', 'currency',
                                                   'delivery_method',
                                                   'fb_published',
                                                   'has_analytics',
                                                   'has_header', 'has_logo',
                                                   'listed', 'payout_type',
                                                   'show_map', 'venue_country',
                                                   'venue_state',
                                                   'payee_name_in_org_name',
                                                   'venue_country_is_source_country',
                                                   'pop_country'])])),
                ('smoter', SMOTE(n_jobs=-1)),
                ('cls', RandomForestClassifier(n_jobs=-1))])

In [545]:
test.predict_proba(X_train)

array([[1.  , 0.  ],
       [0.99, 0.01],
       [0.98, 0.02],
       ...,
       [0.97, 0.03],
       [0.99, 0.01],
       [0.99, 0.01]])

In [546]:
test.transform(X_train)

AttributeError: 'RandomForestClassifier' object has no attribute 'transform'

In [548]:
X_train

,approx_payout_date,body_length,channels,country,currency,delivery_method,description,email_domain,event_created,event_end,event_published,event_start,fb_published,gts,has_analytics,has_header,has_logo,listed,name,name_length,num_order,num_payouts,object_id,org_desc,org_facebook,org_name,org_twitter,payee_name,payout_type,previous_payouts,sale_duration,sale_duration2,show_map,ticket_types,user_age,user_created,user_type,venue_address,venue_country,venue_latitude,venue_longitude,venue_name,venue_state
2481,1381206600,834,11,US,USD,0.0,"<p><span style=""font-size: small; font-family:...",streetsoccerusa.org,1376520728,1380774600,1.377106e+09,1380765600,1,426.98,0,NaN,1,y,"""Kicking It"" Screening",22,22,0,7868367,"<p><span style=""color: #333333; font-family: '...",0.0,Street Soccer USA,0.0,Street Soccer USA,CHECK,"[{'name': 'Street Soccer USA', 'created': '201...",42.0,49,1,"[{'event_id': 7868367, 'cost': 10.0, 'availabi...",0,1376520727,3,601 Van Ness Avenue,US,37.781614,-122.420835,Opera Plaza Cinema,CA
1066,1316757600,822,6,US,USD,0.0,"<p><img alt="""" width=""581"" height=""507"" /></p...",gmail.com,1315172591,1316325600,1.315173e+09,1316311200,1,317.05,0,0.0,1,y,Tonight:Complementary hors d'oeuvres and cockt...,107,8,0,2131786,,0.0,,0.0,,ACH,"[{'name': '', 'created': '2011-10-03 01:10:46'...",13.0,13,0,"[{'event_id': 2131786, 'cost': 0.0, 'availabil...",41,1311610553,1,1310 Sw 2nd Ct,US,26.119185,-80.158972,,
9016,1294063200,389,13,US,USD,0.0,<p>WHAT: Soccer Lock In </p>\r\n<p>WHO: TSC an...,cox.net,1292541080,1293631200,1.292599e+09,1293609300,0,54.74,0,0.0,0,n,TSC HURRICANE LOCK IN,22,2,0,1130939,,0.0,TSC HURRICANE,0.0,TSC,CHECK,"[{'name': 'TSC ', 'created': '2011-01-03 01:10...",11.0,12,1,"[{'event_id': 1130939, 'cost': 25.0, 'availabi...",0,1292541079,1,5817 S 118th East Ave,US,36.077675,-95.843146,Soccer City,OK
1823,1363912200,13275,5,GB,GBP,1.0,"<p class=""MsoNormal"" style=""margin: 0cm 0cm 10...",thinkpositive.co.uk,1361725875,1363480200,1.361726e+09,1363449600,1,2339.19,0,0.0,1,y,Recycle Your Cash one day workshop....with Kev...,55,5,2,5606292,,0.0,,0.0,,ACH,"[{'name': '', 'created': '2013-01-22 00:00:00'...",20.0,20,1,"[{'event_id': 5606292, 'cost': 297.0, 'availab...",1328,1246970051,4,Oxford Street,GB,53.475622,-2.242076,,
12976,1379718000,841,0,GB,GBP,0.0,<p>The Day will include the Stagshead Young En...,hampoloclub.com,1376906892,1379286000,1.376907e+09,1379275200,0,111.89,0,0.0,1,y,Autumn Cup,10,5,45,7919401,,17.0,Ham Polo Club,12.0,,ACH,"[{'name': '', 'created': '2011-05-06 01:17:40'...",27.0,27,1,"[{'event_id': 7919401, 'cost': 5.0, 'availabil...",1174,1275477452,1,Petersham Road,GB,51.446278,-0.301913,Ham Polo Club,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12217,1346644800,15863,6,US,USD,0.0,"<p style=""line-height: 18px; margin-bottom: 1....",gmail.com,1342469917,1346212800,1.342470e+09,1346203800,0,159.24,0,0.0,1,y,Improv 101 - Introduction to Scenework (Tuesdays),49,1,6,3921574,"<p style=""border: 0px; outline: 0px; font-size...",0.0,Alchemy Comedy,0.0,,ACH,"[{'name': '', 'created': '2011-10-26 01:20:14'...",43.0,43,1,"[{'event_id': 3921574, 'cost': 150.0, 'availab...",326,1314318097,3,1 East Coffee Street,US,34.851614,-82.398389,Coffee Underground Theater,SC
4643,1338523200,1057,8,CA,CAD,1.0,"<div><span style=""font-size: 13px; vertical-al...",gmail.com,1332750026,1338091200,1.332751e+09,1338080400,0,878.29,0,1.0,1,y,Student Trustee Alumni Leadership Gala,38,10,0,3219765,,0.0,Alumni Network of the Ontario Student Trustees...,0.0,,ACH,"[{'name': '', 'created': '2012-05-31 03:13:25'...",61.0,61,1,"[{'event_id': 3219765, 'cost': 100.0, 'availab...",655,1276204082,4,955 Bay St.,CA,43.664462,-79.387164,Sutton Place Hotel,Ontario
1304,1363933800,231,5,US,USD,0.0,<p>Come out and help us celebrate 20 years of ...,gmail.com,1359727430,1363501800,1.359731e+09,1363460400,0,3716.

In [549]:
#Grid searching:

In [566]:
X_train_processed = eng_feats.transform(X_train)
X_train_processed = trans.transform(X_train_processed)
X_train_processed

KeyboardInterrupt: 

In [570]:
X_train_processed_sm, y_train_sm = pipe[:-1].fit_resample(X_train, y_train)

In [572]:
X_train_processed_sm.shape

(20870, 575)

In [577]:
grid = [{'cls': [RandomForestClassifier(n_jobs=-1)],
         'cls__n_estimators': [50, 100, 300, 500, 1000, 2000, 3000],
        'cls__min_samples_split': [2, 3, 5, 10]},
       {'cls': [LogisticRegression(n_jobs=-1)],
       'cls__C': np.logspace(-10, 10, 10)},
       {'cls': [xgb.XGBClassifier()]}]
grid_search_pipe = Pipeline(steps=[
    ('cls', RandomForestClassifier(n_jobs=-1))
])

search = GridSearchCV(grid_search_pipe, grid, scoring='f1', n_jobs=-1, cv=5, error_score='raise')
search.fit(X_train_processed_sm, y_train_sm)
search.best_estimator_

Pipeline(steps=[('cls', RandomForestClassifier(n_estimators=3000, n_jobs=-1))])

In [578]:
search.best_params_

{'cls': RandomForestClassifier(n_estimators=3000, n_jobs=-1),
 'cls__min_samples_split': 2,
 'cls__n_estimators': 3000}

In [579]:
search.best_score_

0.995487362213732

In [3]:
#interpreting model

In [40]:
num_cols = ['body_length', 'description', 'name_length', 'num_payouts', 'user_age', 
            'avg_ticket_price', 'total_tickets_available', 'number_of_payouts', 
            'avg_quantity_sold', 'avg_cost_per_ticket', 'days_to_event', 'event_length']

cat_cols = ['country', 'currency', 'delivery_method', 'fb_published', 'has_analytics', 
            'has_header', 'has_logo', 'listed', 'payout_type', 'show_map', 'venue_country', 
            'venue_state', 'payee_name_in_org_name', 'venue_country_is_source_country', 'pop_country']

pipe = joblib.load('../../models/prediction_pipeline.joblib')
pipe.named_steps['transformer'].named_transformers_['categorical_transformer'].named_steps['imputer'].indicator_.features_


array([ 0,  2,  5,  8, 10, 11])

In [15]:
pipe

Pipeline(steps=[('fe',
                 FunctionTransformer(func=<function engineer_features at 0x7fdc134cc0e0>)),
                ('transformer',
                 ColumnTransformer(transformers=[('numeric_transformer',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(add_indicator=True)),
                                                                  ('std',
                                                                   StandardScaler())]),
                                                  ['body_length', 'description',
                                                   'name_length', 'num_payouts',
                                                   'user_age',
                                                   'avg_ticket_price',
                                                   'total_tickets...
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['country', 'currency',
                                                   'delivery_method',
                                                   'fb_published',
                                                   'has_analytics',
                                                   'has_header', 'has_logo',
                                                   'listed', 'payout_type',
                                                   'show_map', 'venue_country',
                                                   'venue_state',
                                                   'payee_name_in_org_name',
                                                   'venue_country_is_source_country',
                                                   'pop_country'])])),
                ('smoter', SMOTE(n_jobs=-1)),
                ('cls', RandomForestClassifier(n_estimators=1000, n_jobs=-1))])

In [51]:
feats = pipe.named_steps['transformer'].named_transformers_['categorical_transformer'].named_steps['imputer'].indicator_.features_
for missing_feature_name in np.array(cat_cols)[feats]:
    cat_cols.append(missing_feature_name + '_is_missing')


In [55]:
cat_col_names = pipe.named_steps['transformer'].named_transformers_['categorical_transformer'].named_steps['onehot'].get_feature_names(cat_cols)

In [58]:
cat_col_names

array(['country_A1', 'country_AE', 'country_AR', 'country_AT',
       'country_AU', 'country_BE', 'country_BG', 'country_BS',
       'country_CA', 'country_CH', 'country_CI', 'country_CM',
       'country_CN', 'country_CO', 'country_CR', 'country_CZ',
       'country_DE', 'country_DK', 'country_DZ', 'country_EC',
       'country_ES', 'country_FI', 'country_FR', 'country_GB',
       'country_GH', 'country_GR', 'country_HR', 'country_HU',
       'country_ID', 'country_IE', 'country_IL', 'country_IM',
       'country_IN', 'country_IS', 'country_IT', 'country_JE',
       'country_JM', 'country_KE', 'country_KH', 'country_LB',
       'country_MA', 'country_MX', 'country_MY', 'country_NA',
       'country_NG', 'country_NI', 'country_NL', 'country_NZ',
       'country_PH', 'country_PK', 'country_PR', 'country_PS',
       'country_QA', 'country_RO', 'country_RS', 'country_RU',
       'country_SE', 'country_SG', 'country_SI', 'country_TH',
       'country_TR', 'country_US', 'country_UY', 'count

In [62]:
all_features = np.array(num_cols + list(cat_col_names))
all_features

array(['body_length', 'description', 'name_length', 'num_payouts',
       'user_age', 'avg_ticket_price', 'total_tickets_available',
       'number_of_payouts', 'avg_quantity_sold', 'avg_cost_per_ticket',
       'days_to_event', 'event_length', 'country_A1', 'country_AE',
       'country_AR', 'country_AT', 'country_AU', 'country_BE',
       'country_BG', 'country_BS', 'country_CA', 'country_CH',
       'country_CI', 'country_CM', 'country_CN', 'country_CO',
       'country_CR', 'country_CZ', 'country_DE', 'country_DK',
       'country_DZ', 'country_EC', 'country_ES', 'country_FI',
       'country_FR', 'country_GB', 'country_GH', 'country_GR',
       'country_HR', 'country_HU', 'country_ID', 'country_IE',
       'country_IL', 'country_IM', 'country_IN', 'country_IS',
       'country_IT', 'country_JE', 'country_JM', 'country_KE',
       'country_KH', 'country_LB', 'country_MA', 'country_MX',
       'country_MY', 'country_NA', 'country_NG', 'country_NI',
       'country_NL', 'country_NZ',

In [69]:
feature_importances = pipe[-1].feature_importances_
order = np.argsort(feature_importances)[::-1]

for idx, (feature_name, importance) in enumerate(zip(all_features[order], feature_importances[order])):
    print(feature_name, importance)
    if idx==24:
        break

description 0.15306405968106013
number_of_payouts 0.12365830185093644
avg_quantity_sold 0.09338724040704767
days_to_event 0.0709321440793196
user_age 0.05636468057704335
num_payouts 0.05509939512379661
payout_type_is_missing_True 0.033096940804429625
avg_ticket_price 0.030181645610023587
payout_type_is_missing_False 0.029246103653878745
delivery_method_0.0 0.028494262079489154
delivery_method_1.0 0.026973359552408833
body_length 0.023451802624348816
avg_cost_per_ticket 0.018271856295347937
name_length 0.017332342676136817
payout_type_ACH 0.01393132959666587
total_tickets_available 0.01187602838736735
payout_type_CHECK 0.011626545632474538
venue_country_is_source_country_True 0.009195204969724268
venue_country_is_source_country_False 0.008209999788791647
has_logo_0 0.007981377494860989
has_logo_1 0.007960631931024215
venue_country_is_missing_True 0.00788624449381738
has_header_is_missing_True 0.007159067880764278
has_header_is_missing_False 0.007086144354936895
venue_country_is_missing_

In [14]:
my_stops = ['0pt', '1pt', '2pt', '4pt', '10pt', '12pt', '14pt', '15pt',
            '0px', '1px', '2px', '4px', '10px', '12px', '14px', '15px',
            'rgb', '255', '0', 'li', 'div', 'u', 'b', '0001pt', '39', '51'
            'meta', 'font', 'size', 'arial', 'nbsp', 'align', 'justify',
            'href', 'style', 'quot', 'msonormal', 'serif', 'text', 'ldquo',
            'rdquo', 'height', 'text', 'mso', 'san', 'margin', 'class', 'tab',
            'roman', 'times', 'http', 'www', 'html', 'background', 'pad',
            'bidi', 'color', 'bidi', 'san', 'rsquo', 'br', 'spin', 'letter',
            'spacing', 'space', 'hyphenate', 'place', 'line', 'placename',
            'placetype', 'border', 'box', 'normal', 'com', 'url', 'link',
            'publish', 'lsdexception', '00', '000', '000000', 'river',
            'family', 'water', 'boat', 'stay', 'helvetica', 'st', 'inherit',
            'width', 'false', 'face', 'non', '51', 'say', 'raft', 'rapid',
            'year', '1', '2', '3', 'rescue', 'true', 'paddle', 'w',
            'lock', 'priority', 'accent', 'semihidden', 'unhidewhenused',
            'table', 'list', 'lock', 'semihidden', 'amp', 'bt', 'grid',
            'layout', 'mode', 'narrative', 'initial', 'variant', 'weight',
            'outline', 'baseline', 'datum', 'vertical', 'leave', 'image',
            'max', 'position', 'display', '68', 'https', 'right', 'ligature',
            'stockticker', '08', '11', '06', '12', 'pa', 'source', '11pt',
            'large', 'march', 'tira', 'niyhwk', 'tcenter', 'posr', 'jim',
            'georgia', 'lucas', 'posr', 'mark', 'get', 'rock', 'be', 'kayker',
            'time', 'ndn', 'thumbtitle', 'thumbnail', 'sliderthumbnailoverlay',
            'neacato', '07', 'witness', 'stockticker', '4', '5', '6', '7',
            'jpg', '300w', 'neue', 'lucida', 'header', 'segoe', 'byline',
            'at4', '75em', '400', '1rem', 'and', 'let', 'near', 'new', '\xa0 ',
            'h1', 'eacute', 'center', '13px', 'rd', 'de', 'center' '800000',
            '\x1f', 'ltr', 'verdana', '-PRON-', '17', '4c4c4c', '17 40625px',
            '11px 17', 'lt', '11px', 'p', 'span', ' ']

seperators = ['.', ';', ':', '/', '&', '=', '(', ')', '-', ',', '>', '<', '_',
              '{', '}']

htmls = ['\\', '\r', '\n', '\t']

spaces = [' '*i for i in range(1, 6)]

nlp = English()
nlp.Defaults.stop_words |= set(my_stops)

def tokenize_and_lemmatize(text):
    text = str(text).lower()
    for sep in seperators:
        text = text.replace(sep, ' ')
    for html in htmls:
        text = text.replace(html, '')
    for space in spaces:
        text = text.replace(space, ' ')
    doc = nlp(text)
    words = []
    for token in doc:
        lemma = token.lemma_
        if not token.is_stop and not token.is_punct and lemma not in my_stops:
            words.append(lemma)
    return words

In [15]:
y = df['acct_type'].str.contains('fraud')
X = df.drop('acct_type', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, stratify=y)

In [16]:
nlp_pipe = Pipeline(steps=[
    ('vec', CountVectorizer(ngram_range=(1,2),
                            max_df=0.9,
                            max_features=10000,
                            token_pattern=None,
                            tokenizer=tokenize_and_lemmatize)),
    ('bayes', MultinomialNB())
])
nlp_pipe

Pipeline(steps=[('vec',
                 CountVectorizer(max_df=0.9, max_features=10000,
                                 ngram_range=(1, 2), token_pattern=None,
                                 tokenizer=<function tokenize_and_lemmatize at 0x7f9685b6c830>)),
                ('bayes', MultinomialNB())])

In [17]:
nlp_pipe.fit(X_train['description'], y_train)

Pipeline(steps=[('vec',
                 CountVectorizer(max_df=0.9, max_features=10000,
                                 ngram_range=(1, 2), token_pattern=None,
                                 tokenizer=<function tokenize_and_lemmatize at 0x7f9685b6c830>)),
                ('bayes', MultinomialNB())])

In [18]:
nlp_pipe.predict_proba(X_train['description'])

array([[7.39074418e-005, 9.99926093e-001],
       [1.00000000e+000, 3.74256212e-105],
       [1.00000000e+000, 5.63155384e-096],
       ...,
       [9.99766361e-001, 2.33638725e-004],
       [9.99999712e-001, 2.87514210e-007],
       [1.00000000e+000, 4.18244802e-038]])